In [52]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


In [54]:
df = pd.read_excel('Items_List_Table.xlsx')
df

,transaction_ID,Item1,Item2,Item3,Item4,Item5
0,0,Laptop,Printer,Tablet,Headset,NaN
1,1,Printer,Monitor,Tablet,NaN,NaN
2,2,Laptop,Printer,Tablet,Headset,NaN
3,3,Laptop,Monitor,Tablet,Headset,NaN
4,4,Printer,Monitor,Tablet,Headset,NaN
5,5,Printer,Tablet,Headset,NaN,NaN
6,6,Monitor,Tablet,NaN,NaN,NaN
7,7,Laptop,Printer,Monitor,NaN,NaN
8,8,Laptop,Tablet,Headset,NaN,NaN
9,9,Printer,Tablet,NaN,NaN,NaN


One-hot encoding

In [ ]:
df_items = [item for item in df.stack().dropna() if not isinstance(item, (int, float))] 

df_encoded = pd.DataFrame({item: df.apply(lambda row: item in row.values, axis=1).astype(int) for item in df_items})
df_encoded

,Laptop,Printer,Tablet,Headset,Monitor
0,1,1,1,1,0
1,0,1,1,0,1
2,1,1,1,1,0
3,1,0,1,1,1
4,0,1,1,1,1
5,0,1,1,1,0
6,0,0,1,0,1
7,1,1,0,0,1
8,1,0,1,1,0
9,0,1,1,0,0


In [57]:
# Run the apriori algorithm
minsup = 0.5

frequent_itemsets = apriori(df_encoded, min_support=minsup, use_colnames=True)
rules = association_rules(frequent_itemsets)

frequent_itemsets

C:\Users\jedre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.5,(Laptop)
1,0.7,(Printer)
2,0.9,(Tablet)
3,0.6,(Headset)
4,0.5,(Monitor)
5,0.6,"(Printer, Tablet)"
6,0.6,"(Headset, Tablet)"


Universal code for a chosen itemset size

In [61]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.5)

rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))

specify_rules = rules[   
    (rules['antecedent_len'] == 1) 
    & (rules['consequents_len'] >= 1) 
    & (rules['confidence'] >= 0.5) # change the MINCONF
    & (rules['lift'] >= 0) ## change the MINLIFT
    ]

specify_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']] 

,antecedents,consequents,support,confidence,lift
0,(Printer),(Tablet),0.6,0.857143,0.952381
1,(Tablet),(Printer),0.6,0.666667,0.952381
2,(Headset),(Tablet),0.6,1.000000,1.111111
3,(Tablet),(Headset),0.6,0.666667,1.111111
